Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [5]:
from itertools import product
from random import random, randint, shuffle, seed
import numpy as np
from scipy import sparse

In [6]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

### THIS IS THE UPDATED VERSION OF MY PREVIOUS WORK. IN THIS PART I TRY TO FIX CORRECTED VERSION OF MY FITNESS CALLS.

In [47]:
'''
With the below parameters we want to optimize a covering problem.
'''


num_points = [100, 1_000, 5_000]
density = [.3, .7]

num = 5_000
density = .7

x = make_set_covering_problem(num, num, density)
print("Element at row=2 and column=2:", x[2, 2])

Element at row=2 and column=2: True


In [59]:
'''
The below functions are original functions implemented by professor.
However, I needed to change the code that it could be compatible with
sparse numpy arrays.
'''


def tweak(state, problem_size):
    new_state = state.copy()
    index = np.random.randint(0, problem_size - 1)
    new_state.remove(index) if index in new_state else new_state.append(index)
    return new_state

def fitness(state):
    global count
    cost = len(state)
    valid = np.unique(np.any(x[state]).nonzero()[1]).shape[0]
    count+=1
    return valid, -cost

In [18]:
'''
The following function optimizes the covering problem.
'''


def optimization(current_state, num, curr_fit_val=(0,0), steps=10_000, show_log=False):
    


    for step in range(steps):
        new_state = tweak(current_state, num)
        fit_val = fitness(new_state)
        if fit_val >= curr_fit_val:
            current_state = new_state
            curr_fit_val = fit_val                
            if show_log:
                print(f'Fitness call: {count}')
                print(f'New state fitness: {fit_val}\n')
                
                
    return current_state


In [19]:
'''
The below results are simplest results with tweaking one value each at a time.

The results for 1000 problem size and 0.3 density will be:

Fitness call (first time covered) : 24
Fitness call (Best solution size) : 1111
Best solution achived             : (1000, -15)
'''


current_state = []
count = 0
optimization(current_state, steps=10_000,num=x.shape[0], show_log=True)

Fitness call: 1
New state fitness: (298, -1)

Fitness call: 2
New state fitness: (497, -2)

Fitness call: 3
New state fitness: (625, -3)

Fitness call: 4
New state fitness: (734, -4)

Fitness call: 5
New state fitness: (803, -5)

Fitness call: 6
New state fitness: (855, -6)

Fitness call: 7
New state fitness: (904, -7)

Fitness call: 8
New state fitness: (933, -8)

Fitness call: 9
New state fitness: (949, -9)

Fitness call: 10
New state fitness: (963, -10)

Fitness call: 11
New state fitness: (975, -11)

Fitness call: 12
New state fitness: (982, -12)

Fitness call: 13
New state fitness: (990, -13)

Fitness call: 14
New state fitness: (995, -14)

Fitness call: 15
New state fitness: (998, -15)

Fitness call: 17
New state fitness: (999, -16)

Fitness call: 19
New state fitness: (1000, -17)

Fitness call: 866
New state fitness: (1000, -16)

Fitness call: 1003
New state fitness: (1000, -15)

Fitness call: 1588
New state fitness: (1000, -14)



[609, 906, 271, 718, 42, 740, 575, 501, 155, 720, 635, 298, 848, 975]

In [20]:
'''
Now what if we define a random one dimmensional window and we start the current state with
those parameters?
'''


win_size = 15
window = [randint(0, num-1) for p in range(0, win_size)]

In [45]:
'''
With windowing we got better results.

Fitness call (first time covered) : 7
Fitness call (Best solution size) : 1102
Best solution achived             : (1000, -13)
'''



current_state = window
count=0
opt_state = optimization(current_state, curr_fit_val=fitness(current_state), steps=10_000,num=x.shape[0], show_log=True)

Fitness call: 2
New state fitness: (4979, -16)

Fitness call: 3
New state fitness: (4984, -17)

Fitness call: 4
New state fitness: (4987, -18)

Fitness call: 5
New state fitness: (4993, -19)

Fitness call: 6
New state fitness: (4998, -20)

Fitness call: 8
New state fitness: (4999, -21)

Fitness call: 15
New state fitness: (5000, -22)

Fitness call: 121
New state fitness: (5000, -21)

Fitness call: 1192
New state fitness: (5000, -20)

Fitness call: 2088
New state fitness: (5000, -19)



In [25]:
'''
Now we start to analyzing the hill climbing method. In hill climbing we have
the maximum number of calls but near optimal results.
'''


def hill_climbing(x):
    hc_states = []
    while fitness(hc_states)[0]!=x.shape[0]:
        current_val=0
        val_list = hc_states.copy()
        for ins in range(x.shape[0]):
            if ins not in hc_states:
                val_list.append(ins)
                val,_ = fitness(val_list)
                if val>current_val:
                    index=ins
                    current_val=val
                val_list.remove(ins)
        
        hc_states.append(index)
    return hc_states



In [26]:
count = 0
hc_states = hill_climbing(x)
print(f'Fitness call: {count}')
print(f'New state fitness: {fitness(hc_states)}\n')

Fitness call: 9966
New state fitness: (1000, -10)



In [52]:
'''
I tried to modify the hill_climbing function to have better cost. 
I added a random window instead of whole solution size. We have better
performance compared to previous methods
'''




def hill_climbing_windowed(x, win_size, fitness_count=0):
    hc_states = []
    while fitness(hc_states)[0]!=x.shape[0]:
        current_val=0
        val_list = hc_states.copy()
        val_window = [randint(0, x.shape[0]-1) for p in range(0, win_size)]
        for ins in val_window:
            if ins not in hc_states:
                val_list.append(ins)
                val,_ = fitness(val_list)
                if val>current_val:
                    index=ins
                    current_val=val
                val_list.remove(ins)
        
        hc_states.append(index)
    return hc_states

In [62]:
count=0
hc_states = hill_climbing_windowed(x, win_size=10)
print(f'Fitness call: {count}')
print(f'New state fitness: {fitness(hc_states)}\n')

Fitness call: 67
New state fitness: (5000, -6)



In [64]:
'''
In this part, I tried to modify tweak method, I add two features inti this method:
    
    
    1- Poping randomly from current state (to increase the exploration rate and avoiding local optimums)
    2- Creating a random index vector from original indexes. Choosing the indexes that have lower similarity
       with existing indexes (based on how many intersection they have.)
       
       
Unfortunatly, this method adds extra time which not worth it. However, we obtain better results
compared to two first methods.
'''


def tweak_modified(state, problem_size, add_step=2, pop_num=1, win_size=30):
    new_state = state.copy()
    shuffle(new_state)
    if pop_num<len(new_state):
        for _ in range(pop_num): new_state.pop()
        
    if len(new_state)==0:    
        index = np.random.randint(0, problem_size - 1)

        
    val_window = [randint(0, x.shape[0]-1) for p in range(0, win_size)]
    min_diff = np.inf
    for i in val_window:
        for n in new_state:
            intsec = np.intersect1d(np.unique(np.any(x[[n]]).nonzero()[1]),np.unique(np.any(x[[i]]).nonzero()[1]))
            if intsec.shape[0] < min_diff:
                min_diff = intsec.shape[0]
                index = i


    new_state.remove(index) if index in new_state else new_state.append(index)   

    return new_state

In [63]:
current_state = []
count = 0
print(f'Fitness call before tweaking: {count}')
print(f'New state fitness tweaking: {fitness(hc_states)}\n')


for step in range(10_000):
    new_state = tweak_modified_new(current_state, num, pop_num=0, win_size=30)
    count+=1
    if fitness(new_state) >= fitness(current_state):
        current_state = new_state
        print(f'Fitness call: {count}')
        print(f'New state fitness: {fitness(new_state)}\n')

Fitness call before tweaking: 0
New state fitness tweaking: (5000, -6)



NameError: name 'tweak_modified_new' is not defined

### Now we start analyzing all results together. We use three main methods:

    1- Method provided by professor
    2- Greedy hill climbing
    3- Optimized hill climbing (my method)

In [67]:
num_points = [(10,100), (10,1_000), (10,5_000)] # first element of the tuples is window size
density_list = [.3, .7]
methods = ['basic', 'windowed_hc']

'''
Basic:   Professor method (tweaking from initial empty list)
Mine :   Windowed hill climbing
'''

for num in num_points:
    for density in density_list:
        for method in methods:
            print(f'Method        : {method}\nDensity       : {density}\nProblem size  : {num[1]}\n')
            
            count=0
            x = make_set_covering_problem(num[1], num[1], density)
            hc_states = hill_climbing(x)
            print(f'Greedy fitness call: {count}')
            print(f'Greedy optimum: {fitness(hc_states)}\n')
            
            count=0
            if method=='basic':
                current_state = []
            if method=='windowed_hc':
                current_state = hill_climbing_windowed(x, win_size=num[0])
                print(f'Fitness call: {count}')
                print(f'New state fitness: {fitness(current_state)}\n')
                
            # Calculating greedy optimums with hill hill climbing

            opt_state = optimization(current_state, num=x.shape[0], show_log=True)
            
            print('______________________________________________________________________')
        

Method        : basic
Density       : 0.3
Problem size  : 100

Greedy fitness call: 592
Greedy optimum: (100, -6)

Fitness call: 1
New state fitness: (34, -1)

Fitness call: 2
New state fitness: (55, -2)

Fitness call: 3
New state fitness: (70, -3)

Fitness call: 4
New state fitness: (78, -4)

Fitness call: 6
New state fitness: (82, -5)

Fitness call: 7
New state fitness: (87, -6)

Fitness call: 8
New state fitness: (95, -7)

Fitness call: 9
New state fitness: (97, -8)

Fitness call: 10
New state fitness: (99, -9)

Fitness call: 12
New state fitness: (100, -10)

Fitness call: 14
New state fitness: (100, -9)

______________________________________________________________________
Method        : windowed_hc
Density       : 0.3
Problem size  : 100

Greedy fitness call: 592
Greedy optimum: (100, -6)

Fitness call: 85
New state fitness: (100, -8)

Fitness call: 87
New state fitness: (100, -9)

Fitness call: 101
New state fitness: (100, -8)

__________________________________________________